In [3]:
import math
from scipy.optimize import fsolve
import numpy as np

# Bubble Point Calculation

In [4]:
class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

class Bubble_Point:
    """
    Calculator for bubble point temperature if pressure is passed and
    bubble point pressure if temperature is passed.
    """
    def __init__(self, composition):
        self.key_split_fraction = 0   # all split fractions are 0 for bubble point
        self.composition = composition
        key_component = 0
        # find total feed flow and check for most abundant species
        self.total_feed = 0
        for i in range(len(composition)):
            self.total_feed += composition[i][-1]
            if composition[i][-1] > composition[key_component][-1]:
                key_component = i
        # for bubble point, n = k'
        self.key_component = key_component
        self.k_prime = key_component

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature to find bubble point pressure.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure to find bubble point temperature.

        Iterate on bubble point temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for bubble point temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated bubble point temperature.
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition, temperature)
        self.get_relative_volatility()
        average_volatility_helper = lambda component: component.relative_volatility * component.fk / self.total_feed
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_calc_temperature(self, pressure):
        """
        Calculate the bubble point temperature with the bubble point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Bubble point temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime = k_prime_component.relative_volatility * pressure / self.average_volatility
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the bubble point pressure with the bubble point equation.

        Returns:
            float: Bubble point pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  self.average_volatility * k_prime_component.vap_pressure / k_prime_component.relative_volatility

# Dew Point

In [5]:
class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

class Dew_Point:
    """
    Calculator for dew point temperature if pressure is passed and
    dew point pressure if temperature is passed.
    """
    def __init__(self, composition):
        self.key_split_fraction = 1   # all split fractions are 1 for dew point
        self.composition = composition
        key_component = 0
        # find total feed flow and check for most abundant species
        self.total_feed = 0
        for i in range(len(composition)):
            self.total_feed += composition[i][-1]
            if composition[i][-1] > composition[key_component][-1]:
                key_component = i
        # for dew point, n = k'
        self.key_component = key_component
        self.k_prime = key_component

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature to find dew point pressure.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure to find dew point temperature.

        Iterate on dew point temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for dew point temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated dew point temperature.
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate Kn (zk / relative_volatility)

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition, temperature)
        self.get_relative_volatility()
        # sum zk / alpha_k/n for each component
        kn_helper = lambda component: (component.fk / self.total_feed) / component.relative_volatility
        self.kn = sum(map(kn_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_calc_temperature(self, pressure):
        """
        Calculate the dew point temperature with the dew point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Dew point temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime =  pressure * self.kn
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the dew point pressure with the dew point equation.

        Returns:
            float: Dew point pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  k_prime_component.vap_pressure / self.kn

# Flash Case 1
### split fraction and temperature/pressure specified

In [6]:
class Component:
    """
    Represents a pure component within the flash mixture.
    """
    def __init__(self, A, B, C, fk, temperature=None, vap_pressure=None):
        # Antoine equation constants
        self.A = A
        self.B = B
        self.C = C
        # feed molar flow
        self.fk = fk
        # use Antoine to calculate p_vap or T based on the inputs of the Component
        self.temperature = temperature if temperature else self.antoine_equation(vap_pressure=vap_pressure)
        self.vap_pressure = vap_pressure if vap_pressure else self.antoine_equation(temperature=temperature)

    def antoine_equation(self, temperature=None, vap_pressure=None):
        """
        Antoine equation to calculate temperature or vapour pressure of the pure component.

        Args:
            temperature (float): temperature of the pure component in K
            vap_pressure (float): vapour pressure of the pure component in mmHg
        Returns:
            float: temperature in K or vapour pressure in mmHg depending on method input
        """
        if temperature:
            return math.exp(self.A - self.B / (self.C + temperature))
        return self.B /(self.A - math.log(vap_pressure)) - self.C

    def set_relative_volatility(self, key_vap_pressure):
        """
        Calculate the relative volitility with the vapour pressure of the key component.

        Args:
            key_vap_pressure (float): Vapour pressure of the key component
        """
        self.relative_volatility = self.vap_pressure / key_vap_pressure

    def set_phase_flows(self, key_split_fraction):
        """
        Calculate the split fraction, liquid phase molar flow, and vapour phase molar flow.

        Args:
            key_split_fraction (float): Split fraction of the key component
        """
        self.split_fraction = self.relative_volatility * key_split_fraction / ((self.relative_volatility - 1)* key_split_fraction + 1)
        self.vapour_flow = self.split_fraction * self.fk
        self.liquid_flow = (1 - self.split_fraction) * self.fk

    def set_phase_composition(self, total_liquid, total_vapour):
        """
        Calculate the liquid phase mole fraction and vapour phase mole fraction.

        Args:
            total_liquid (float): Total liquid phase molar flow of the mixture
            total_vapour (float): Total vapour phase molar flow of the mixture
        """
        self.liquid_fraction = self.liquid_flow / total_liquid
        self.vapour_fraction = self.vapour_flow / total_vapour

class Flash_Case_1:
    """
    Case 1 flash calculator: key component split fraction and temperature/pressure specified.
    """
    def __init__(self, composition, key_component, key_split_fraction):
        self.key_component = key_component
        self.key_split_fraction = key_split_fraction
        self.composition = composition

    def specify_temperature(self, temperature):
        """
        Run flash calculations at a specified temperature.

        Args:
            temperature (float): Mixture temperature in K
        """
        self.flash_intermediate_steps(temperature)
        self.calc_pressure = self.get_calc_pressure()

    def specify_pressure(self, pressure, t_guess):
        """
        Run flash calculations at a specified pressure.

        Iterate on flash temperature with fsolve based on an initial guess.

        Args:
            pressure (float): Mixture pressure in mmHg
            t_guess (float): Inital guess for temperature in K
        """
        self.pressure = pressure
        self.calc_temperature = fsolve(self.temperature_iteration, t_guess)[0]

    def temperature_iteration(self, temperature):
        """
        Iterative function for fsolve to optimize that runs flash calculations.

        Args:
            temperature (float): Mixture temperature in K

        Returns:
            float: The difference between guessed and calculated temperature
        """
        self.flash_intermediate_steps(temperature)
        return temperature - self.get_calc_temperature(self.pressure)

    def flash_intermediate_steps(self, temperature):
        """
        Run flash calculation algorithm with helper functions.

        1. Initialize components at a given temperature
        2. Calculate relative volatility for each component
        3. Calculate the molar flows in the liquid and vapour phases for each component
        4. Calculate the mole fractions in the liquid and vapour phases for each component
        5. Calculate the average volatility

        Args:
            temperature (float): Mixture temperture in K
        """
        self.components = self.init_components(self.composition,temperature)
        self.get_relative_volatility()
        self.get_phase_flows()
        self.get_phase_composition()
        average_volatility_helper = lambda component: component.relative_volatility * component.liquid_fraction
        self.average_volatility = sum(map(average_volatility_helper,self.components))

    def init_components(self, composition, temperature):
        """
        Create a new Component object for each item in the composition list.

        Args:
            composition (list): list of lists containing component Antoine coefficients and feed amount
            temperature (float): temperature of the mixture in K
        Returns:
            list[Component]: list of pure Component objects in the mixture
        """
        components = []
        for component in composition:
            A, B, C, fk = component
            components.append(Component(A=A, B=B, C=C, fk=fk, temperature=temperature))
        return components

    def get_relative_volatility(self):
        """
        Calculate the relative volatility of each component with the vapour pressure
        of the key component.
        """
        key_vap_pressure = self.components[self.key_component].vap_pressure
        for component in self.components:
            component.set_relative_volatility(key_vap_pressure)

    def get_phase_flows(self):
        """
        Calculate the vapour and liquid flows of the mixture and each component.
        Determine the K prime component.
        """
        self.liquid_flow = 0
        self.vapour_flow = 0
        curr_k_prime = 0
        for i in range(len(self.components)):
            component = self.components[i]
            component.set_phase_flows(self.key_split_fraction)
            self.liquid_flow += component.liquid_flow
            self.vapour_flow += component.vapour_flow
            # check for k prime, most abundant species in the liquid phase
            if component.liquid_flow > self.components[curr_k_prime].liquid_flow:
                curr_k_prime = i
        self.k_prime = curr_k_prime

    def get_phase_composition(self):
        """
        Calculate the vapour and liquid mole fractions of each component.
        """
        for component in self.components:
            component.set_phase_composition(self.liquid_flow, self.vapour_flow)

    def get_calc_temperature(self, pressure):
        """
        Calculate the flash temperature with the bubble point equation.

        Args:
            pressure (float): Mixture pressure in mmHg
        Returns:
            float: Flash temperature in K from the antoine equation of the K prime component
        """
        k_prime_component = self.components[self.k_prime]
        p_vap_k_prime = k_prime_component.relative_volatility * pressure / self.average_volatility
        return k_prime_component.antoine_equation(vap_pressure=p_vap_k_prime)

    def get_calc_pressure(self):
        """
        Calculate the flash pressure with the bubble point equation.

        Returns:
            float: Flash pressure in mmHg
        """
        k_prime_component = self.components[self.k_prime]
        return  self.average_volatility * k_prime_component.vap_pressure / k_prime_component.relative_volatility

## Linear Mass Balances

### EL

In [26]:
# mu1, mu2, mu31, mu32, mu41, mu42, mu6, mu71, mu72, mu81, mu82, mu91, mu92
EL = np.array([
              [1, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0],
              [-0.93, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.985, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.015, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.979, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.021, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])
b_EL = np.array([96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

x_EL = np.linalg.solve(EL,b_EL)
x_EL

array([1371.42857143, 1275.42857143, 1256.29714286,   19.13142857,
       1229.91490286,   26.38224   ,   45.51366857,   45.51366857,
          0.        ,   45.51366857,    0.        ,    0.        ,
          0.        ])

### EA

In [27]:
# mu1, mu2, mu31, mu32, mu41, mu42, mu6, mu71, mu72, mu81, mu82, mu91, mu92
EA = np.array([
              [1, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0],
              [-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.121, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.879, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.01, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.99, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -0.995, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -0.005, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -0.005, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -0.995, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.995, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.005, 0, 1]
])

b_EA = np.array([0, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

x_EA = np.linalg.solve(EA,b_EA)
x_EA

array([ 0.59687015, 96.59687015, 11.68822129, 84.90864886,  0.11688221,
       11.57133908, 96.47998794, 95.997588  ,  0.48239994,  0.47998794,
       95.51760006, 95.04001206,  0.477588  ])

### DEE

In [28]:
# mu1, mu2, mu31, mu32, mu41, mu42, mu6, mu71, mu72, mu81, mu82, mu91, mu92
DEE = np.array([
              [1, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.24, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.76, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -0.995, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -0.005, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])

DEE_reaction = 5 * (x_EA[1] ** 2) / (0.53 * x_EL[0])
b_DEE = np.array([0, DEE_reaction, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

x_DEE = np.linalg.solve(DEE,b_DEE)
x_DEE

array([63.90457335, 64.18699613, 32.09349807, 32.09349807,  7.70243954,
       24.39105853, 56.4845566 , 56.4845566 ,  0.        , 56.20213381,
        0.28242278,  0.28242278,  0.        ])

### Water

In [29]:
# mu1, mu2, mu31, mu32, mu41, mu42, mu6, mu71, mu72, mu81, mu82, mu91, mu92
W = np.array([
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.054, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, -0.946, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -1.9E-3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, -0.998, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -0.1, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, -0.9, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1]
])

W_feed = x_EL[0] * 0.6
W_reaction = x_EL[0] * 0.53
W_azeotrope = x_EA[-2] /0.854 * 0.145

b_W = np.array([W_feed, W_reaction, 0, 0, 0, 0, 0, 0, 0, 0, 0, W_azeotrope, 0])

x_W = np.linalg.solve(W,b_W)
x_W

array([8.22857143e+02, 7.26857143e+02, 3.92502857e+01, 6.87606857e+02,
       7.45755429e-02, 3.91717851e+01, 7.26778642e+02, 7.26778642e+01,
       6.54100778e+02, 0.00000000e+00, 7.26778642e+01, 1.61367702e+01,
       5.65410940e+01])

## Temperature/Pressure Calc

### Constants

In [11]:
E2E_PRESSURE = 68.5 * 750.06    # convert pressure to mmHg
E2E_COMPONENT_NAMES = ["ethylene", "diethyl-ether", "ethanol", "water"]
# pure component: [A, B, C, fk]
E2E_COMPOSITION = np.array([
                        [6.74756, 585.0, 255.0],
                        [7.4021, 1391.4, 273.16],
                        [8.04494, 1554.3, 222.65],
                        [8.10765, 1750.286, 235.0]])

# convert A and B from log10 to ln
E2E_COMPOSITION[:,:2] *= np.log(10)
# convert C from Celsius to Kelvin
E2E_COMPOSITION[:, 2] -= 273.15
E2E_COMPOSITION

array([[ 1.55368311e+01,  1.34701228e+03, -1.81500000e+01],
       [ 1.70439651e+01,  3.20381690e+03,  1.00000000e-02],
       [ 1.85241589e+01,  3.57890801e+03, -5.05000000e+01],
       [ 1.86685540e+01,  4.03018245e+03, -3.81500000e+01]])

### Combined Stream Flows

In [12]:
# mu1, mu2, mu31, mu32, mu41, mu42, mu6, mu71, mu72, mu81, mu82, mu91, mu92
stream_flows = np.column_stack((x_EL, x_DEE, x_EA, x_W))
mu03 = 2.25E-2 * np.sum(stream_flows[2])
mu03, stream_flows

(30.134905828326904,
 array([[1.37142857e+03, 6.39045733e+01, 5.96870153e-01, 8.22857143e+02],
        [1.27542857e+03, 6.41869961e+01, 9.65968702e+01, 7.26857143e+02],
        [1.25629714e+03, 3.20934981e+01, 1.16882213e+01, 3.92502857e+01],
        [1.91314286e+01, 3.20934981e+01, 8.49086489e+01, 6.87606857e+02],
        [1.22991490e+03, 7.70243954e+00, 1.16882213e-01, 7.45755429e-02],
        [2.63822400e+01, 2.43910585e+01, 1.15713391e+01, 3.91717851e+01],
        [4.55136686e+01, 5.64845566e+01, 9.64799879e+01, 7.26778642e+02],
        [4.55136686e+01, 5.64845566e+01, 9.59975880e+01, 7.26778642e+01],
        [0.00000000e+00, 0.00000000e+00, 4.82399940e-01, 6.54100778e+02],
        [4.55136686e+01, 5.62021338e+01, 4.79987940e-01, 0.00000000e+00],
        [0.00000000e+00, 2.82422783e-01, 9.55176001e+01, 7.26778642e+01],
        [0.00000000e+00, 2.82422783e-01, 9.50400121e+01, 1.61367702e+01],
        [0.00000000e+00, 0.00000000e+00, 4.77588000e-01, 5.65410940e+01]]))

### 3. Flash

In [30]:
# feed is stream 2
composition_3 = np.column_stack((E2E_COMPOSITION, stream_flows[1]))
composition_3
unit_3 = Flash_Case_1(composition=composition_3, key_component=1, key_split_fraction=0.5)
unit_3.specify_pressure(pressure=E2E_PRESSURE, t_guess=310)
temp_3 = unit_3.calc_temperature
temp_3

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


464.41366045139495

### 4.1 Absorber Vapour

In [14]:
composition_41 = np.column_stack((E2E_COMPOSITION, stream_flows[4]))
dew_41 = Dew_Point(composition=composition_41)
dew_41.specify_pressure(pressure=(68*750.06), t_guess=temp_3)
temp_41 = dew_41.calc_temperature
temp_41

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


326.0665040463248

### 4.2 Absorber Liquid

In [15]:
composition_42 = np.column_stack((E2E_COMPOSITION, stream_flows[5]))
bubble_42 = Bubble_Point(composition=composition_42)
bubble_42.specify_pressure(pressure=(68*750.06), t_guess=temp_3)
temp_42 = bubble_42.calc_temperature
temp_42

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


410.5416367455376

### 7.1 Dewatering Column Condenser

In [16]:
# assume total condenser, T_cond = 310 K
t_cond =310
composition_71 = np.column_stack((E2E_COMPOSITION, stream_flows[7]))
bubble_71 = Bubble_Point(composition=composition_71)
bubble_71.specify_temperature(temperature=t_cond)
pressure_71 = bubble_71.calc_pressure / 750.06   # convert from mmHg to bar
pressure_71

12.706693514560197

### 7.2 Dewatering Column Reboiler

In [17]:
# assuming a pressure drop of 0.5 bar across the dewatering column:
pressure_72 = (pressure_71  + 0.5) * 750.06   # convert back to mmHg
composition_72 = np.column_stack((E2E_COMPOSITION, stream_flows[8]))
dew_72 = Dew_Point(composition=composition_72)
dew_72.specify_pressure(pressure=pressure_72, t_guess=310)
dew_72.calc_temperature

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


463.81197720673873

### 8.1 De-ethering Column Condenser

In [18]:
# assume total condenser, T_cond = 310 K
t_cond = 310
composition_81 = np.column_stack((E2E_COMPOSITION, stream_flows[9]))
dew_81 = Dew_Point(composition=composition_81)
dew_81.specify_temperature(temperature=t_cond)
pressure_81 = dew_81.calc_pressure / 750.06   # convert from mmHg to bar
pressure_81

1.8517241911776094

### 8.2 De-ethering Column Reboiler

In [19]:
# assuming a pressure drop of 0.5 bar across the dewatering column:
pressure_82 = (pressure_81  + 0.5) * 750.06   # convert back to mmHg
composition_82 = np.column_stack((E2E_COMPOSITION, stream_flows[10]))
bubble_82 = Bubble_Point(composition=composition_82)
bubble_82.specify_pressure(pressure=pressure_82, t_guess=310)
bubble_82.calc_temperature

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


382.30604050352537

### 9.1 Finishing Column Condenser

In [20]:
# assuming P_cond is 1 bar
pressure_91 = 750.06   # convert to mmHg
composition_91 = np.column_stack((E2E_COMPOSITION, stream_flows[11]))
bubble_91 = Bubble_Point(composition=composition_91)
bubble_91.specify_pressure(pressure=pressure_91, t_guess=310)
bubble_91.calc_temperature

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


353.1234555480373

### 9.2 Finishing Column Reboiler

In [21]:
# assuming a pressure drop of 0.5 bar across the dewatering column:
pressure_92 = 1.5 * 750.06   # convert to mmHg
composition_92 = np.column_stack((E2E_COMPOSITION, stream_flows[12]))
dew_92 = Dew_Point(composition=composition_92)
dew_92.specify_pressure(pressure=pressure_92, t_guess=310)
dew_92.calc_temperature

<ipython-input-6-a8b3a33947ed>:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return math.exp(self.A - self.B / (self.C + temperature))


384.1635285860576

### 6. Mixer

In [22]:
# use a weighted average of stream temperatures to estimate mu6 temp
mu32 = np.sum(stream_flows[3])
mu42 = np.sum(stream_flows[5])
mu6 = mu32 + mu42
temp_6 = (mu32 * temp_3 + mu42 * temp_42) / mu6
temp_6

458.50298267477405

### 1. Mixer

In [23]:
# use a weighted average of stream temperatures to estimate mu1 temp
mu01 = 96
mu02 = 0.6 * stream_flows[0,0]  # feed condition for water
inlet_temp = 300
temp_81 = 310   # condenser temp
mu81 = np.sum(stream_flows[9])
mu41 = np.sum(stream_flows[4])
mu1 = np.sum(stream_flows[0])
temp_1 = ((mu01 + mu02) * inlet_temp + mu41 * temp_41 + mu81 * temp_81) / mu1
temp_1

314.74670977693324